# 계층 구현하기
- 오차역전파를 계산 그래프로 이해하였다
- 신경망에서 계층이라는 것은 하나의 기능 단위로, 신경망에서 각각 하나의 클래스로 구현하기에
- 계산 그래프의 덧셈노드, 곱셈노드를 계층 단위로, 각각 하나의 클래스로 구현한다

## 곱셈 계층

In [9]:
class MulLayer:
    def __init__ (self):
        self.x = None
        self.y = None
        
    def forward(self,x,y):
        self.x = x
        self.y = y
        out = x*y
        return out
    
    def backward(self,dout):
        dx = dout * self.y # x 와 y를 바꿔서 dx를 구한다
        dy = dout * self.x
        return dx,dy

계산 그래프
- 문제 : 100원짜리 사과를 2개 샀는데 소비세 10%가 붙는 문제

In [13]:
apple = 100 ; apple_num = 2 ; tax = 1.1
mul_apple_layer = MulLayer() # 개수에 따른 총액이 계산되는 곱셈노드
mul_tax_layer = MulLayer() # 소비세와 최종금액이 계산되는 곱셈노드

# 순전파
apple_price = mul_apple_layer.forward(apple,apple_num)
price = mul_tax_layer.forward(apple_price,tax)

print(int(price))

220


In [11]:
# 역전파
dprice = 1 # 미분값 입력신호가 1
d_apple_price,dtax = mul_tax_layer.backward(dprice)
d_apple,d_apple_num = mul_apple_layer.backward(d_apple_price)

print("사과가격에 대한 총액 변화량",d_apple_price)
print("소비세에 대한 총액 변화량",dtax)
print("사과값에 대한 총액 변화량",d_apple)
print("사과개수에 대한 총액 변화량",d_apple_num)

사과가격에 대한 전체가격 변화량 1.1
소비세에 대한 전체가격 변화량 200
사과값에 대한 전체가격 변화량 2.2
사과개수에 대한 전체가격 변화량 110.00000000000001


## 덧셈 계층

In [12]:
class AddLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self,x,y):
        self.x = x
        self.y = y
        return x+y
    
    def backward(self,dout):
        dy = dout * 1
        dx = dout * 1
        return dx,dy

계산 그래프
- 문제 : 100원짜리 사과를 2개와 150원짜리 귤 3개를 샀는데 소비세 10%가 붙는 문제

In [15]:
apple = 100 ; orange = 150 ; apple_num = 2 ; orange_num = 3 ; tax = 1.1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
mul_add_layer = AddLayer()
mul_tax_layer = MulLayer() 

# 순전파
apple_price = mul_apple_layer.forward(apple,apple_num) # 사과금액 구하기
orange_price = mul_orange_layer.forward(orange,orange_num) # 오렌지금액 구하기

add_price = mul_add_layer.forward(apple_price,orange_price) # 과일총액 = 사과금액 + 오렌지금액

price = mul_tax_layer.forward(add_price,tax) # 총액 = 과일총액 * 소비세
print(int(price))

715


In [17]:
# 역전파

dprice = 1 

d_add_price, dtax = mul_tax_layer.backward(dprice) # 곱셈노드 역전파
d_apple_price, d_orange_price = mul_add_layer.backward(d_add_price) # 덧셈노드 역전파 그대로 흘러나감

d_apple, d_apple_num = mul_apple_layer.backward(d_apple_price) # 곱셈노드 역전파
d_orange,d_orange_num = mul_orange_layer.backward(d_orange_price) # 곱셈노드 역전파

print("사과가격에 대한 총액 변화량",d_apple_price)
print("사과값에 대한 총액 변화량",d_apple)
print("사과개수에 대한 총액 변화량",d_apple_num)

print("오렌지가격에 대한 총액 변화량",d_orange_price)
print("오렌지값에 대한 총액 변화량",d_orange)
print("오렌지개수에 대한 총액 변화량",d_orange_num)

print("소비세에 대한 총액 변화량",dtax)

사과가격에 대한 총액 변화량 1.1
사과값에 대한 총액 변화량 2.2
사과개수에 대한 총액 변화량 110.00000000000001
오렌지가격에 대한 총액 변화량 1.1
오렌지값에 대한 총액 변화량 3.3000000000000003
오렌지개수에 대한 총액 변화량 165.0
소비세에 대한 총액 변화량 650
